In [1]:
from urllib2 import urlopen
from json import load
import pandas as pd
from pandas.io.json import json_normalize
import itertools

In [2]:
##api_key = 'MDIzNDI5NDMzMDE0NTkwMTI5ODY1NzQ4Ng000'

In [3]:
url = 'http://api.npr.org/query?apiKey=MDIzNDI5NDMzMDE0NTkwMTI5ODY1NzQ4Ng000&numResults=3&format=json&id=1002'

In [4]:
response = urlopen(url)
json_obj = load(response)

In [5]:
##the next several cells are test code to practice mapping to the information we want in the df -- commented out for now

In [6]:
'''
for story in json_obj['list']['story']:

    print 'ID: ' + story['id']
    
    print 'TITLE: ' + story['title']['$text']

    print 'DATE: ' + story['storyDate']['$text'] + '\n'
'''

"\nfor story in json_obj['list']['story']:\n\n    print 'ID: ' + story['id']\n    \n    print 'TITLE: ' + story['title']['$text']\n\n    print 'DATE: ' + story['storyDate']['$text'] + '\n'\n"

In [7]:
'''for story in json_obj['list']['story']:
    for parent in story['parent']:
        print story['id'], parent['title']['$text']
'''

"for story in json_obj['list']['story']:\n    for parent in story['parent']:\n        print story['id'], parent['title']['$text']\n"

In [8]:
'''for story in json_obj['list']['story']:
    print story['id'], story['text']['paragraph']
'''
'''      
for story in json_obj['list']['story']:
    for text in story['text']['paragraph']:
        print story['id'], text['$text']
'''

"      \nfor story in json_obj['list']['story']:\n    for text in story['text']['paragraph']:\n        print story['id'], text['$text']\n"

In [9]:
## Build the data frame by creating a list of dictionaries, then converting the list of dictionaries into a data frame

##First data frame: one row per distinct story category

dicts_list = []

In [10]:
for story in json_obj['list']['story']:
    for parent in story['parent']:
        d = {
            'id': story['id'],
            'title': story['title']['$text'].encode('ascii'),
            'category': parent['title']['$text'].encode('ascii')    
        }
        dicts_list.append(d)

In [11]:
df = pd.DataFrame(dicts_list, columns=('id', 'title', 'category'))

In [13]:
df.head(5)

,id,title,category
0,478441023,Clinton Holds Narrow Lead Over Sanders In Kent...,Politics
1,478441023,Clinton Holds Narrow Lead Over Sanders In Kent...,Elections
2,478441023,Clinton Holds Narrow Lead Over Sanders In Kent...,Politics
3,478441023,Clinton Holds Narrow Lead Over Sanders In Kent...,U.S.
4,478441023,Clinton Holds Narrow Lead Over Sanders In Kent...,Home Page Top Stories
5,478441023,Clinton Holds Narrow Lead Over Sanders In Kent...,News
6,478417091,Sanders Doubles Down On Nevada Convention Cont...,Politics
7,478417091,Sanders Doubles Down On Nevada Convention Cont...,Politics
8,478417091,Sanders Doubles Down On Nevada Convention Cont...,News
9,478417091,Sanders Doubles Down On Nevada Convention Cont...,sanders campaign


In [14]:
##Second data frame: one row per story, with one column containing a list of all the story's categories

dicts_reshape = []

In [15]:
for story in json_obj['list']['story']:
    categories_list = []
    d = {
        'id': story['id'],
        'title': story['title']['$text'].encode('ascii')
        }
    for parent in story['parent']:
        category = parent['title']['$text'].encode('ascii')
        categories_list.append(category)   
        d['category'] = categories_list
        d['top_category'] = []
    dicts_reshape.append(d)

In [16]:
df_reshape = pd.DataFrame(dicts_reshape, columns=('id', 'title', 'category', 'top_category'))

In [17]:
df_reshape

,id,title,category,top_category
0,478441023,Clinton Holds Narrow Lead Over Sanders In Kent...,"[Politics, Elections, Politics, U.S., Home Pag...",[]
1,478417091,Sanders Doubles Down On Nevada Convention Cont...,"[Politics, Politics, News, sanders campaign, s...",[]
2,478410141,Older Voters To Candidates: Don't Forget About Us,"[Elections, Home Page Top Stories, Politics, A...",[]


In [18]:
df_reshape['category'][0]

['Politics', 'Elections', 'Politics', 'U.S.', 'Home Page Top Stories', 'News']

In [19]:
##determine most popular categories
all_categories = df['category'].tolist()

In [20]:
category_counter = {}  

In [21]:
for category in all_categories:
    if category in category_counter:
        category_counter[category] += 1
    else:
        category_counter[category] = 1

In [22]:
popular_categories = sorted(category_counter, key = category_counter.get, reverse = True)

In [23]:
top_5 = popular_categories[:5]

In [24]:
top_5

['Politics',
 'Elections',
 'U.S.',
 'Home Page Top Stories',
 'NPR Stories For Apple News']

In [25]:
##filter category lists to retain only top categories

In [31]:
for index, row in df_reshape.iterrows():
    compressed_category = [x for x in row['category'] if x in top_5]
    df_reshape['top_category'][index] = compressed_category

In [32]:
df_reshape

,id,title,category,top_category
0,478441023,Clinton Holds Narrow Lead Over Sanders In Kent...,"[Politics, Elections, Politics, U.S., Home Pag...","[Politics, Elections, Politics, U.S., Home Pag..."
1,478417091,Sanders Doubles Down On Nevada Convention Cont...,"[Politics, Politics, News, sanders campaign, s...","[Politics, Politics, NPR Stories For Apple New..."
2,478410141,Older Voters To Candidates: Don't Forget About Us,"[Elections, Home Page Top Stories, Politics, A...","[Elections, Home Page Top Stories, Politics, N..."


In [33]:
##d3 export:  create a list of pairs of relationships to convert to json

pairs_d3 = []

In [34]:
for category_list in df_reshape['top_category']:
    for pair in itertools.combinations(category_list, 2):
        pairs_d3.append(pair)

In [35]:
df_d3 = pd.DataFrame(pairs_d3, columns=('source', 'target'))

In [36]:
df_d3

,source,target
0,Politics,Elections
1,Politics,Politics
2,Politics,U.S.
3,Politics,Home Page Top Stories
4,Elections,Politics
5,Elections,U.S.
6,Elections,Home Page Top Stories
7,Politics,U.S.
8,Politics,Home Page Top Stories
9,U.S.,Home Page Top Stories


In [38]:
df_d3.to_csv('links.csv')

In [45]:
nodes = pd.DataFrame()

In [46]:
nodes['node'] = top_5

In [47]:
nodes

,node
0,Politics
1,Elections
2,U.S.
3,Home Page Top Stories
4,NPR Stories For Apple News


In [48]:
nodes.to_csv('nodes.csv')